This tutorial can be downloaded [link](https://github.com/west-code-development/westpy/raw/master/doc/tutorials/westpy_300.ipynb).

# 3.0 RESTful Tutorials

We will show here how to interact with the remote server where the `pw.x`, `wstat.x` and `wfreq.x` are already installed.

We start by importing WESTPy and generating a ground state input file for methane.

In [1]:
from westpy import *


 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  3.1.1
Today            :  2018-09-19 15:33:45.581889


In [2]:
geom = Geometry()

In [3]:
geom.setCell((25,0,0),(0,25,0),(0,0,25))

In [4]:
geom.addAtomsFromOnlineXYZ( "https://west-code.org/doc/training/methane/CH4.xyz" )

In [5]:
geom.addSpecies( "C", "http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf")
geom.addSpecies( "H", "http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.0.upf")

In [6]:
gs = GroundState(geom,xc="PBE",ecut=40.0)

In [7]:
gs.generateInputPW()


Generated file:  pw.in


We now create a session on the remote server.

In [8]:
session = Session("your.email@domain.edu")

Check the inbox/spam folder of your email and click on the link to activate the session


In [10]:
session.status()

{'Active': 'True', 'Time Left': '59 minutes'}

In [11]:
download = ["http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf","http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.0.upf"]
pw_output = session.run("pw","pw.in","pw.out",download,3)

Requested to run executable pw on server imedevel.uchicago.edu
Generated  pw.out


We now generate the input for `wstat.x`.

In [12]:
data = {}
data["input_west"] = {}
data["input_west"]["qe_prefix"] = "calc"
data["input_west"]["west_prefix"] = "calc"
data["input_west"]["outdir"] = "./"
data["wstat_control"] = {}
data["wstat_control"]["wstat_calculation"] = "S"
data["wstat_control"]["n_pdep_eigen"] = 5

writeJsonFile("wstat.in",data)


File written :  wstat.in


In [13]:
wstat_output = session.run("wstat","wstat.in","wstat.out",download,3)

Requested to run executable wstat on server imedevel.uchicago.edu
Generated  wstat.out


We now generate the input for `wfreq.x`.

In [14]:
data["wfreq_control"] = {}
data["wfreq_control"]["wfreq_calculation"] = "XWGQ"
data["wfreq_control"]["n_pdep_eigen_to_use"] = 5
data["wfreq_control"]["qp_bandrange"] = [1,5]
data["wfreq_control"]["n_refreq"] = 300
data["wfreq_control"]["ecut_refreq"] = 2.0

writeJsonFile("wfreq.in",data)


File written :  wfreq.in


In [16]:
wfreq_output = session.run("wfreq","wfreq.in","wfreq.out",download,3)

Requested to run executable wfreq on server imedevel.uchicago.edu
Generated  wfreq.out


We now stop the session.

In [17]:
session.stop()

{'Success': 'Session removed and cleared'}